# 시계열 머신러닝용 음식물쓰레기 데이터 전처리
- 알수없음 지역을 지운 newdata_음식물쓰레기.csv 와 알수없음 데이터를 merge

### 1)데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
foods = pd.read_csv('./rowdata/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV', encoding='CP949')
foods

c:\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,base_date,city,emd_cd,emd_nm,em_area_cd,em_cnt,em_g,pay_amt
0,2018-01-01,제주시,50110670,외도동,W6X062,2,15500,464
1,2018-01-01,제주시,50110630,아라동,W6XA97,25,59800,1787
2,2018-01-01,제주시,50110630,아라동,W6XA96,8,25350,758
3,2018-01-01,제주시,50110630,아라동,W6XA95,12,30000,898
4,2018-01-01,제주시,50110630,아라동,W6XA94,14,44050,1317
...,...,...,...,...,...,...,...,...
2211440,2021-06-30,제주시,50110250,한림읍,W6XEB4,12,58250,1743
2211441,2021-06-30,제주시,50110250,한림읍,W6XEB7,15,111000,3326
2211442,2021-06-30,제주시,50110250,한림읍,W6XEB8,21,48300,1444
2211443,2021-06-30,제주시,50110250,한림읍,W6XEA2,5,22050,661


### 2) em_g=0 인 행을 지우고 emd_nm= '알수없는' 행 불러오기

In [ ]:
non = foods[foods['emd_nm']=='알수없음']

In [ ]:
non = non[non['em_g']!=0]

In [ ]:
non['region']='알수없음'

In [ ]:
non_g = non.groupby(['base_date','emd_nm','region','emd_cd']).sum()
non_g

,,,,em_cnt,em_g,pay_amt
base_date,emd_nm,region,emd_cd,,,
2018-01-01,알수없음,알수없음,알수없음,41,398950,11960
2018-01-02,알수없음,알수없음,알수없음,39,271600,8137
2018-01-03,알수없음,알수없음,알수없음,32,301650,9041
2018-01-04,알수없음,알수없음,알수없음,27,289850,8689
2018-01-05,알수없음,알수없음,알수없음,35,253150,7588
...,...,...,...,...,...,...
2021-06-26,알수없음,알수없음,알수없음,11,57850,1733
2021-06-27,알수없음,알수없음,알수없음,10,48450,1451
2021-06-28,알수없음,알수없음,알수없음,13,63050,1887


### 3) pay_amt 칼럼 삭제

In [ ]:
non_g = non_g.drop(['pay_amt'],axis=1)

In [ ]:
non_g = non_g.reset_index()

### 4) 알수 없음 지역중 없는 날짜 넣기

In [ ]:
dt_index = pd.date_range(start='20180101', end='20210630')
base_date = [];
for date in dt_index:
       base_date.append(date)
datecheck = pd.DataFrame(base_date)
datecheck

,0
0,2018-01-01
1,2018-01-02
2,2018-01-03
3,2018-01-04
4,2018-01-05
...,...
1272,2021-06-26
1273,2021-06-27
1274,2021-06-28
1275,2021-06-29


In [ ]:
datecheck = datecheck.rename(columns={0:'base_date'})
datecheck['base_date'] = datecheck['base_date'].astype(str)

In [ ]:
fornon = pd.merge(datecheck,non_g,how='outer',on='base_date')
fornon

### 5) em_cnt, em_g 값중 NaN에 평균값 넣기

In [ ]:
cnt_avg =np.round( non_g['em_cnt'].mean(),2)
cnt_avg

27.66

In [ ]:
g_avg =np.round( non_g['em_g'].mean(),0)
g_avg

165679.0

In [ ]:
fornon['em_cnt'] = fornon['em_cnt'].fillna(cnt_avg)

In [ ]:
fornon[fornon['em_g'].isnull()]

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
395,2019-01-31,NaN,NaN,NaN,27.66,NaN
396,2019-02-01,NaN,NaN,NaN,27.66,NaN
397,2019-02-02,NaN,NaN,NaN,27.66,NaN
398,2019-02-03,NaN,NaN,NaN,27.66,NaN
399,2019-02-04,NaN,NaN,NaN,27.66,NaN
...,...,...,...,...,...,...
920,2020-07-09,NaN,NaN,NaN,27.66,NaN
921,2020-07-10,NaN,NaN,NaN,27.66,NaN
922,2020-07-11,NaN,NaN,NaN,27.66,NaN
923,2020-07-12,NaN,NaN,NaN,27.66,NaN


In [ ]:
fornon['em_g'] = fornon['em_g'].fillna(g_avg)
fornon[fornon['em_g'].isnull()]

,base_date,emd_nm,region,emd_cd,em_cnt,em_g


In [ ]:
fornon[fornon['emd_nm'].isnull()]

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
395,2019-01-31,NaN,NaN,NaN,27.66,165679.0
396,2019-02-01,NaN,NaN,NaN,27.66,165679.0
397,2019-02-02,NaN,NaN,NaN,27.66,165679.0
398,2019-02-03,NaN,NaN,NaN,27.66,165679.0
399,2019-02-04,NaN,NaN,NaN,27.66,165679.0
...,...,...,...,...,...,...
920,2020-07-09,NaN,NaN,NaN,27.66,165679.0
921,2020-07-10,NaN,NaN,NaN,27.66,165679.0
922,2020-07-11,NaN,NaN,NaN,27.66,165679.0
923,2020-07-12,NaN,NaN,NaN,27.66,165679.0


### 6) emd_nm, emd_cd, region 에 알수없음 넣기

In [ ]:
fornon.fillna('알수없음',inplace=True)

In [ ]:
fornon[fornon['base_date']=='2019-04-04']

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
458,2019-04-04,알수없음,알수없음,알수없음,27.66,165679.0


In [ ]:
fornon

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,알수없음,알수없음,알수없음,41.0,398950.0
1,2018-01-02,알수없음,알수없음,알수없음,39.0,271600.0
2,2018-01-03,알수없음,알수없음,알수없음,32.0,301650.0
3,2018-01-04,알수없음,알수없음,알수없음,27.0,289850.0
4,2018-01-05,알수없음,알수없음,알수없음,35.0,253150.0
...,...,...,...,...,...,...
1272,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
1273,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
1274,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
1275,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


### 7) 알수없음 dataframe 과 음식물쓰레기 데이터 merge

In [ ]:
foods = pd.read_csv("./datas/newdata_음식물쓰레기.csv",encoding='CP949')
foods

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,50130253.0,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,50130259.0,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,50130320.0,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,50130250.0,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,50130310.0,77.0,296200.0
...,...,...,...,...,...,...
49602,2021-06-30,이도이동,제주시_중,50110540.0,4032.0,7304500.0
49603,2021-06-30,이도일동,제주시_중,50110530.0,90.0,139750.0
49604,2021-06-30,이호동,제주시_중,50110680.0,339.0,764450.0
49605,2021-06-30,일도이동,제주시_중,50110520.0,2856.0,4963450.0


In [ ]:
df = pd.concat([foods,fornon])

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,5.01303e+07,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,5.01303e+07,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,5.01303e+07,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,5.01302e+07,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,5.01303e+07,77.0,296200.0
...,...,...,...,...,...,...
1272,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
1273,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
1274,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
1275,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


In [ ]:
df = df.reset_index()
df = df.drop(['index'],axis=1)
df

,level_0,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,0,2018-01-01,남원읍,서귀포시_동,5.01303e+07,304.0,1239600.0
1,1,2018-01-01,성산읍,서귀포시_동,5.01303e+07,90.0,424100.0
2,2,2018-01-01,표선면,서귀포시_동,5.01303e+07,112.0,452850.0
3,3,2018-01-01,대정읍,서귀포시_서,5.01302e+07,309.0,1264950.0
4,4,2018-01-01,안덕면,서귀포시_서,5.01303e+07,77.0,296200.0
...,...,...,...,...,...,...,...
50879,50879,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
50880,50880,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
50881,50881,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
50882,50882,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


In [ ]:
df = df.drop(['level_0'],axis=1)
df

,base_date,emd_nm,region,emd_cd,em_cnt,em_g
0,2018-01-01,남원읍,서귀포시_동,5.01303e+07,304.0,1239600.0
1,2018-01-01,성산읍,서귀포시_동,5.01303e+07,90.0,424100.0
2,2018-01-01,표선면,서귀포시_동,5.01303e+07,112.0,452850.0
3,2018-01-01,대정읍,서귀포시_서,5.01302e+07,309.0,1264950.0
4,2018-01-01,안덕면,서귀포시_서,5.01303e+07,77.0,296200.0
...,...,...,...,...,...,...
50879,2021-06-26,알수없음,알수없음,알수없음,11.0,57850.0
50880,2021-06-27,알수없음,알수없음,알수없음,10.0,48450.0
50881,2021-06-28,알수없음,알수없음,알수없음,13.0,63050.0
50882,2021-06-29,알수없음,알수없음,알수없음,10.0,63400.0


### 8)데이터 저장하기

In [ ]:
df.to_csv("./datas/newdata_시계열용_음식물쓰레기.csv",index=False,encoding='CP949')